In [1]:
import requests
from lxml import etree
from collections import OrderedDict
from urllib.parse import quote
import csv
import traceback
import re, os
from time import sleep
from datetime import datetime
import json
import pandas as pd
import random

In [2]:
def get_text(raw):
    noise_text = ['\n',' ']
    new = ''.join(str(s) for s in raw if s not in noise_text)
    return new

In [3]:
def get_author_info(uid):
    url = f'https://weibo.com/ajax/profile/info?custom={uid}'
                    
    res_author = requests.get(url, headers=headers).text
    author_following, author_followers, author_gender, author_location, author_IPlocation = '','','','',''
    info_dict = json.loads(res_author)['data']['user']
    author_following = info_dict['friends_count']
    author_followers = info_dict['followers_count']
    author_gender = info_dict['gender']
    author_location = info_dict['location']

    return author_following, author_followers, author_gender, author_location

In [4]:
def get_raw_text(weibo):
    iffull = weibo.xpath(".//a[@action-type='fl_unfold']")
    if iffull:
        text_raw = weibo.xpath(".//p[@node-type='feed_list_content_full']")[0] # 'feed_list_content_full' for imcompleted weibos
    else:
        text_raw = weibo.xpath(".//p[@node-type='feed_list_content']")[0]
    return text_raw

In [5]:
def get_time(weibo):
    try:
        time = get_text(weibo.xpath(".//div[@class='from']/a")[0].xpath("string()")).translate(mytable)
    except:
        time = get_text(weibo.xpath(".//p[@class='from']/a")[0].xpath("string()")).translate(mytable)
        
    if time[1].isdigit():
        if int(time[0:2]) < 20:
            time = '2023-'+time
    else:
        time = '2023-05'
    return time

In [6]:
def get_interaction(weibo):
    if len(weibo.xpath(".//a[@action-type='feed_list_like']/em")) == 0:
        like = weibo.xpath(".//div[@class='card-act']//span[@class='woo-like-count']")[0].text.replace('赞','')
        comment = get_text(weibo.xpath(".//a[@action-type='feed_list_comment']")[0].xpath("string()").replace('评论',''))
        repost = get_text(weibo.xpath(".//a[@action-type='feed_list_forward']")[0].xpath("string()").replace('转发',''))
    else: # old version
        like = weibo.xpath(".//a[@action-type='feed_list_like']/em")[0].text
        comment = get_text(weibo.xpath(".//a[@action-type='feed_list_comment']")[0].text).replace('评论','')
        repost = get_text(weibo.xpath(".//a[@action-type='feed_list_forward']")[0].text).replace('转发','')
    return like,comment,repost

In [17]:
#Update cookie
with open('weibo/cookie.txt', 'r') as f:
    Cookie = f.readlines()[0]  # Read all lines in the file
User_Agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15"
headers = {
            'Cookie': Cookie,
            'User-Agent': User_Agent
        }

savefile = 'weibo/weibo.csv'
infofile = 'weibo/info.txt'
if os.path.exists(infofile):
    with open(infofile,'r') as f:
        s = f.read()
        my_dict = json.loads(s)
start_time = my_dict['start_time']
end_time = my_dict['end_time']
keyword = my_dict['keyword']
page = my_dict['page']
mytable = start_time.maketrans('年月日:','----')

In [21]:
continue_scrape = 1

with open(savefile, 'a+') as f:
    header = ['mid','text','time','favorite','like','comment','repost','author_uid','author_url','author_following',
              'author_followers','author_gender','author_location']
    writer = csv.DictWriter(f, delimiter=',', lineterminator='\n',fieldnames=header)
    if not os.path.getsize(savefile):
        writer.writeheader()

    while(continue_scrape):
        while ((page <= 25) & (continue_scrape)):
            url = 'https://s.weibo.com/weibo?q={}&typeall=1&suball=1&timescope=custom%3A{}%3A{}&Refer=g&page={}'.format(quote(keyword),start_time,end_time,page)
            res = requests.get(url=url, headers=headers)

            html = etree.HTML(res.text.encode('utf-8'))

            try:
                weibos = html.xpath("//div[@action-type='feed_list_item']")

                # 自动结束
                if len(weibos) == 0:
#                    continue_scrape = 0
                    print(f'no weibo, current time: {end_time}, page: {page}\n')
                    page += 1
                    continue

                for weibo in weibos:
                    mid = weibo.xpath("./@mid")[0]

                    text_raw = get_raw_text(weibo)
                    text = get_text(text_raw.xpath("string()").replace('收起全文d',''))
                    time = get_time(weibo)
                    if time[0:4] == '2023':
                        print('time: ', time, '. Please check the time')
                        break
                    favorite = ''
                    like,comment,repost = get_interaction(weibo)

                    uid = re.search(r'/(.*)\?refer',weibo.xpath(".//a/@href")[0].replace('//',''), re.IGNORECASE).group(1)
                    author_url = f'https://weibo.com/p/100505{uid}/info?mod=pedit_more'
                    author_following, author_followers, author_gender, author_location = get_author_info(uid)
#                    author_following, author_followers, author_gender, author_location = 0,0,'',''

                    writer.writerow({'mid':mid,'text':text,'time':time,'favorite':favorite,'like':like,'comment':comment,'repost':repost,
                                       'author_uid':uid,'author_url':author_url, 'author_following':author_following, 'author_followers':author_followers, 
                                        'author_gender':author_gender, 'author_location':author_location})
            except:
                print(traceback.format_exc())
            page += 1
            sleep(random.uniform(4, 6))
            with open(infofile, 'w') as f:
                f.write('{"keyword": "女权", "start_time": "'+start_time+'", '+'"end_time": "{}", "page":{}'.format(end_time,page)+'}')
        print(url)
        sleep(10)
        if time[0:4] != '2023':
            end_time = time[0:-3] #be careful about duplicates since its based on hour instead of minute
        page = 1

https://s.weibo.com/weibo?q=%E5%A5%B3%E6%9D%83&typeall=1&suball=1&timescope=custom%3A2021-02-01-00%3A2021-03-24-15&Refer=g&page=25
https://s.weibo.com/weibo?q=%E5%A5%B3%E6%9D%83&typeall=1&suball=1&timescope=custom%3A2021-02-01-00%3A2021-03-24-14&Refer=g&page=25
https://s.weibo.com/weibo?q=%E5%A5%B3%E6%9D%83&typeall=1&suball=1&timescope=custom%3A2021-02-01-00%3A2021-03-24-13&Refer=g&page=25
Traceback (most recent call last):
  File "/var/folders/8v/xpk1jpgx2kbgxh9xl6dwblt00000gp/T/ipykernel_85649/3202781325.py", line 41, in <module>
    author_following, author_followers, author_gender, author_location = get_author_info(uid)
  File "/var/folders/8v/xpk1jpgx2kbgxh9xl6dwblt00000gp/T/ipykernel_85649/1948625335.py", line 6, in get_author_info
    info_dict = json.loads(res_author)['data']['user']
  File "/Users/liaoyufang/opt/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Users/liaoyufang/opt/anaconda3/lib/python3.9/json/decoder.

KeyboardInterrupt: 

In [ ]:
#old version
    # if match_following := re.search(r'<strong class="W_f..">(.*)</strong><span class="S_txt2">关注',res_author, re.IGNORECASE):
    #     author_following = match_following.group(1)
    # if match_followers := re.search(r'关注.*<strong class="W_f..">(.*)</strong><span class="S_txt2">粉丝',res_author, re.IGNORECASE):
    #     author_followers = match_followers.group(1)
    # if match_gender := re.search(r'性别：</span><span class="pt_detail">(.)</span></li>',res_author, re.IGNORECASE):
    #     author_gender = match_gender.group(1)
    # if match_location := re.search(r'所在地：</span><span class="pt_detail">(.*)</span>.*IP属地：',res_author, re.IGNORECASE):
    #     author_location = match_location.group(1)
    # if match_IPlocation := re.search(r'IP属地：</span><span class="pt_detail ipRegion">(.*)<span class="W_icon icon_askS">',res_author, re.IGNORECASE):
    #     author_IPlocation = match_IPlocation.group(1)